In [1]:
!pip install requests

jeton pour l'api: GxVU3F0PVtasjhz4igAtgOMqHjRvu5Aj
jeton pour le jeux de dionnées: af69c4c7f8cf9793bce1aef0b55a83822d4094cfde75ff4e068e809c

D'après la doc technique:

on peut ajouter des paramètres comme format=json pour obtenir du JSON, ou filtrer par status=available pour n’avoir que les lignes actives ou à venir.

In [13]:
import requests

API_TOKEN = "GxVU3F0PVtasjhz4igAtgOMqHjRvu5Aj"

# URL d'appel avec format JSON
url = "https://prim.iledefrance-mobilites.fr/marketplace/ilico/getData?method=getlc&format=json&status=available"

headers = {
    "apikey": API_TOKEN,
    "Accept": "application/json"
}

# Appel à l'API
response = requests.get(url, headers=headers)



In [14]:
# Vérification de la réponse
if response.status_code == 200:
    data = response.json()
    print("Nombre de lignes reçues :", len(data['dataObjects']['CompositeFrame']['frames']['GeneralFrame']))
else:
    print("Erreur", response.status_code, ":", response.text)


Nombre de lignes reçues : 2


In [25]:
lines = []

frames = data['dataObjects']['CompositeFrame']['frames']['GeneralFrame']
print(len(frames))
for frame in frames:
    print(frame['TypeOfFrameRef']['ref'])
    if frame['TypeOfFrameRef']['ref'] == 'FR1:TypeOfFrame:NETEX_LIGNE:':
        lines = frame['members']['Line']
        break

print("Nombre de lignes :", len(lines))
print("Exemple de ligne :", lines[0])


2
FR1:TypeOfFrame:NETEX_COMMUN:
FR1:TypeOfFrame:NETEX_LIGNE:
ok
Nombre de lignes : 2106
Exemple de ligne : {'version': 'any', 'dataSourceRef': 'FR1:OrganisationalUnit:STIF:', 'created': '2014-07-16T00:00:00Z', 'changed': '2025-03-26T08:09:24Z', 'status': 'active', 'id': 'FR1:Line:C00001:', 'ValidBetween': {'FromDate': '2014-07-16T00:00:00Z'}, 'keyList': {'KeyValue': [{'typeOfKey': 'ALTERNATE_IDENTIFIER', 'Key': 'TLB_LIST_ID', 'Value': '222_222_082'}, {'typeOfKey': 'REGULATORY_INFORMATION', 'Key': 'AccessibilityProgram', 'Value': 'true'}]}, 'Extensions': {'ServiceFacilitySet': {'id': 'FR1:ServiceFacilitySet:C00001:', 'version': 'any', 'ClimateControlList': 'other'}}, 'Name': '482', 'ShortName': '482', 'Description': "13/02/2019 Anciennement nommée Centaure. Fusion Centaure et Bord de l'Eau pour former la ligne 2 qui devient le 10/10/2021 la ligne 482.", 'TransportMode': 'bus', 'TransportSubmode': {'BusSubmode': 'demandAndResponseBus'}, 'PrivateCode': '522522482', 'OperatorRef': {'versio

In [19]:
data

{'PublicationTimestamp': '2025-04-07T05:58:09Z',
 'ParticipantRef': 'FR1-LIGNE',
 'dataObjects': {'CompositeFrame': {'version': '2.0',
   'created': '2025-04-07T05:58:09Z',
   'id': 'FR1:CompositeFrame:NETEX_IDF-20250407055809Z:LOC',
   'TypeOfFrameRef': {'ref': 'FR1:TypeOfFrame:NETEX_IDF:',
    'TypeOfFrameRef': 'version="1.04:FR1-NETEX_IDF-2.0"'},
   'frames': {'GeneralFrame': [{'version': '2.0',
      'created': '2025-04-07T05:58:09Z',
      'id': 'FR1:AbstractFrame:NETEX_COMMUN-20250407055809Z:LOC',
      'TypeOfFrameRef': {'ref': 'FR1:TypeOfFrame:NETEX_COMMUN:',
       'TypeOfFrameRef': 'version="1.04:FR1-NETEX_COMMUN-2.1"'},
      'members': {'OrganisationalUnit': [{'version': 'any',
         'id': 'FR1:OrganisationalUnit:STIF:',
         'Name': 'IDFM',
         'TypeOfOrganisationPartRef': {'ref': 'FR1_Organisation'}},
        {'version': 'any',
         'id': 'FR1:OrganisationalUnit:RATPDEV:',
         'Name': 'RATP CAP ',
         'TypeOfOrganisationPartRef': {'ref': 'FR1_Org

In [33]:
def extract_commercial_lines2(data):
    general_frames = data['dataObjects']['CompositeFrame']['frames']['GeneralFrame']
    lignes_commerciales = []
    
    for frame in general_frames:
        if frame['TypeOfFrameRef']['ref'] == 'FR1:TypeOfFrame:NETEX_LIGNE:':
            lignes_commerciales = frame['members']['Line']
            break
    
    extracted_data = []
    for ligne in lignes_commerciales:
        extracted_data.append({
            "id": ligne.get("id"),
            "name": ligne.get("Name"),
            "short_name": ligne.get("ShortName"),
            "status": ligne.get("status"),
            "mode": ligne.get("TransportMode"),
            "submode": ligne.get("TransportSubmode", {}).get("BusSubmode") or ligne.get("TransportSubmode", {}).get("RailSubmode"),
            "public_code": ligne.get("PublicCode"),
            "private_code": ligne.get("PrivateCode"),
            "operator_ref": ligne.get("OperatorRef", {}).get("ref"),
            "from_date": ligne.get("ValidBetween", {}).get("FromDate"),
            "to_date": ligne.get("ValidBetween", {}).get("ToDate")
        })
    
    df = pd.DataFrame(extracted_data)
    return df

    #df.to_csv("lignes_commerciales_idf.csv", index=False)
    #print(" Fichier CSV exporté : lignes_commerciales_idf.csv")

def extract_commercial_lines(data):
    frames = data['dataObjects']['CompositeFrame']['frames']['GeneralFrame']
    
    # Séparation des deux cadres NETEX_COMMUN et NETEX_LIGNE
    operators = []
    lignes_commerciales = []

    for frame in frames:
        if frame['TypeOfFrameRef']['ref'] == 'FR1:TypeOfFrame:NETEX_COMMUN:':
            operators = frame['members'].get('Operator', [])
        elif frame['TypeOfFrameRef']['ref'] == 'FR1:TypeOfFrame:NETEX_LIGNE:':
            lignes_commerciales = frame['members'].get('Line', [])

    operator_dict = {
        op['id']: op.get('Name', '') for op in operators
    }

    extracted_data = []
    for ligne in lignes_commerciales:
        operator_ref = ligne.get("OperatorRef", {}).get("ref")
        operator_name = operator_dict.get(operator_ref, "Inconnu")

        extracted_data.append({
            "id": ligne.get("id"),
            "name": ligne.get("Name"),
            "short_name": ligne.get("ShortName"),
            "status": ligne.get("status"),
            "mode": ligne.get("TransportMode"),
            "submode": ligne.get("TransportSubmode", {}).get("BusSubmode") or ligne.get("TransportSubmode", {}).get("RailSubmode"),
            "public_code": ligne.get("PublicCode"),
            "private_code": ligne.get("PrivateCode"),
            "operator_ref": operator_ref,
            "operator_name": operator_name,
            "from_date": ligne.get("ValidBetween", {}).get("FromDate"),
            "to_date": ligne.get("ValidBetween", {}).get("ToDate")
        })

    df = pd.DataFrame(extracted_data)
    return df

In [34]:
extract_commercial_lines(data)

,id,name,short_name,status,mode,submode,public_code,private_code,operator_ref,operator_name,from_date,to_date
0,FR1:Line:C00001:,482,482,active,bus,demandAndResponseBus,None,522522482,FR1:Operator:1004:LOC,Keolis Ouest Val-de-Marne,2014-07-16T00:00:00Z,None
1,FR1:Line:C00003:,Licorne,Licorne,active,bus,localBus,None,522522481,FR1:Operator:1004:LOC,Keolis Ouest Val-de-Marne,2014-07-16T00:00:00Z,None
2,FR1:Line:C00004:,483,483,active,bus,regionalBus,None,522522483,FR1:Operator:1004:LOC,Keolis Ouest Val-de-Marne,2014-07-16T00:00:00Z,None
3,FR1:Line:C00006:,480,480,active,bus,localBus,None,522522480,FR1:Operator:1004:LOC,Keolis Ouest Val-de-Marne,2014-07-16T00:00:00Z,None
4,FR1:Line:C00007:,282,282,active,bus,localBus,None,522522282,FR1:Operator:1004:LOC,Keolis Ouest Val-de-Marne,2014-07-16T00:00:00Z,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2101,FR1:Line:C02781:,TAD Boissise le Roi - Pringy,TAD,active,bus,demandAndResponseBus,None,None,FR1:Operator:1009:LOC,Transdev Melun,2023-08-18T00:00:00Z,None
2102,FR1:Line:C02782:,4157 (ex 17S),4157,active,bus,regionalBus,4157,520520017,FR1:Operator:1015:LOC,Keolis Val d'Yerres Val de Seine,2025-02-28T00:00:00Z,None
2103,FR1:Line:C02783:,4602,4602,inactive,bus,None,4602,526526023,FR1:Operator:1013:LOC,RD Saclay,2025-04-22T00:00:00Z,None
2104,FR1:Line:C02784:,4621,4621,inactive,bus,None,4621,526526024,FR1:Operator:1013:LOC,RD Saclay,2025-04-22T00:00:00Z,None
